# Data analytics of EXI Dataset

- Data collected from https://european-investor-exchange.com/en/trade-list


## Step 0 - Familiarisation with data
- read csv files of october
- basis statistics
- descriptive statistics
- basic visuals

## Step 1 - Data Cleaning
- Check and summarize missing values per column using combined_df.isnull().sum().
- Drop duplicates with combined_df.drop_duplicates(inplace=True).
- Convert data types:
- Ensure Quantity and Unit Price are numeric.
- Convert Trading day & Trading time UTC to datetime.
- Standardize text-based columns such as Side and Price Currency by stripping spaces and converting to uppercase.

## Step 2 - Feature Engineering
- Add a new column: Trade Amount = Quantity × Unit Price.
- Extract datetime features: date, time, hour, day of week.
- Create log-transformed or normalized columns for skewed variables if relevant to modeling or visualization.

## Step 3 - Exploratory Data Analysis (EDA)
- Plot data distributions for Unit Price, Quantity, and Trade Amount.
- Explore relationships like Quantity vs Unit Price and Trade Amount vs Side.
- Identify time-based trading patterns — daily or hourly volumes and price trends.
- Analyze top instruments and venues by total trade quantity or value.

## Step 4 - Outlier Detection
- Use boxplots or z-score thresholds to highlight outliers in Quantity and Unit Price.
- Optionally apply the interquartile range (IQR) method to filter extreme values.
- Visualize outlier patterns across Side or Venue Identifier.

## Step 5 - Venue & Instrument Analysis
- Aggregate by Venue Identifier and Instrument Identifier to observe trading concentration.
- Identify instruments or venues with the highest variance in price or volume.
- Visualize via bar charts or pie charts the proportion of trading volume by venue/instrument.

## Step 6 - Model Preparation
- Select relevant features — numeric variables (e.g., Trade Amount, Quantity) and categorical ones (Side, Venue Identifier).
- Encode categorical variables using one-hot encoding.
- Address data imbalance or skew using sampling or scaling.
- Split into training and testing sets for modeling.

In [ ]:
# import basis libraries
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# List all CSV files in current directory
csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
print(csv_files)

In [ ]:
# read single file to see the schema of the csv file
df = pd.read_csv('Official_Trade_List_2025-10-01.csv')

# View the first five rows
print(df.head(5))


In [ ]:
# List all CSV files in the current directory
csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]

# Read and combine them into one DataFrame
if csv_files:  # Check if any CSV files exist
    combined_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
    print("Combined DataFrame shape:", combined_df.shape)
else:
    print("No CSV files found in the current directory.")


In [ ]:
combined_df.head(5)

# Basis Overview
- data types
- numerical distributions
- missing values.
- trade price ranges
- the most frequently traded instruments

In [ ]:
combined_df.columns

In [ ]:
combined_df.info()

In [ ]:
print(combined_df.describe())

In [ ]:
print(combined_df.isnull().sum())

### Descriptive statistics
-  trade price ranges
- the most frequently traded instruments.

In [ ]:
print(combined_df.groupby('Side')['Unit Price'].describe())

In [ ]:
print(combined_df['Instrument Identifier'].value_counts().head(5))

### Key Visualizations

In [ ]:
sns.countplot(data=combined_df, x='Side')
plt.title('Distribution of Trade Sides')
plt.show()


In [ ]:
sns.histplot(data=combined_df, x='Unit Price', bins=50, kde=True)
plt.title('Distribution of Unit Prices')
plt.show()

In [ ]:
sns.boxplot(data=combined_df, x='Side', y='Unit Price')
plt.title('Unit Price by Trade Side')
plt.show()

In [ ]:
sns.scatterplot(data=combined_df, x='Quantity', y='Unit Price', hue='Side', alpha=0.6)
plt.title('Quantity vs Unit Price by Side')
plt.show()


In [ ]:
combined_df['Trading day & Trading time UTC'] = pd.to_datetime(combined_df['Trading day & Trading time UTC'])
daily_volume = combined_df.groupby(combined_df['Trading day & Trading time UTC'].dt.date)['Quantity'].sum()

daily_volume.plot(kind='line', figsize=(12,5), title='Daily Trade Volume Over Time')
plt.ylabel('Total Quantity')
plt.show()

## Step-1 Data Cleaning
How this helps:

- Converts all types correctly for quantitative and time-series analysis.
- Normalizes categorical text fields.
- Eliminates duplicates and reveals missing data accurately.

In [ ]:
# 1. Check and summarize missing values per column
print(combined_df.isnull().sum())

# 2. Drop duplicate rows
combined_df.drop_duplicates(inplace=True)

# 3. Convert data types
combined_df['Trading day & Trading time UTC'] = pd.to_datetime(combined_df['Trading day & Trading time UTC'], errors='coerce')
combined_df['Quantity'] = pd.to_numeric(combined_df['Quantity'], errors='coerce')
combined_df['Unit Price'] = pd.to_numeric(combined_df['Unit Price'], errors='coerce')

# 4. Standardize text-based columns
combined_df['Side'] = combined_df['Side'].str.strip().str.upper()
combined_df['Price Currency'] = combined_df['Price Currency'].str.strip().str.upper()

# 5. Summary after cleaning
print('Shape after cleaning:', combined_df.shape)
print('Remaining null values:', combined_df.isnull().sum())


## Step-2 Feature Engineering

- The new Trade Amount column captures trade value and will be key in later analyses.
- Extracted date/time fields allow temporal pattern detection (hourly, daily, weekday trends).
- Log-transformed features help stabilize variance in price and quantity during visualization and modeling.

In [ ]:
# 1. Create new derived column: Trade Amount (Quantity × Unit Price)
combined_df['Trade Amount'] = combined_df['Quantity'] * combined_df['Unit Price']

# 2. Extract datetime components for time-based analysis
combined_df['Trade Date'] = combined_df['Trading day & Trading time UTC'].dt.date
combined_df['Trade Hour'] = combined_df['Trading day & Trading time UTC'].dt.hour
combined_df['Trade Weekday'] = combined_df['Trading day & Trading time UTC'].dt.day_name()
combined_df['Trade Month'] = combined_df['Trading day & Trading time UTC'].dt.month

# 3. (Optional) Log-transform skewed numeric features for better visualization later
import numpy as np
for col in ['Quantity', 'Unit Price', 'Trade Amount']:
    combined_df[f'log_{col}'] = np.log1p(combined_df[col])

# 4. Quick verification summary
print(combined_df[['Quantity', 'Unit Price', 'Trade Amount']].describe())
print(combined_df[['Trade Date', 'Trade Hour', 'Trade Weekday', 'Trade Month']].head())


In [ ]:
combined_df.head(3)

## Step-3 EDA
#### What this step reveals:

- Numerical summaries for trade prices, quantities, and total value.
- Top traded instruments to identify liquidity concentration.
- Side distribution (Buy/Sell ratio).
- Temporal patterns — daily and hourly trading activity.
- Relationships between trade quantity and unit price.

In [ ]:
# 1. Statistical summaries
print("\n[INFO] Descriptive summary of key numeric columns:")
print(combined_df[['Quantity', 'Unit Price', 'Trade Amount']].describe())

print("\n[INFO] Top 10 most traded instruments by total quantity:")
print(combined_df.groupby('Instrument Identifier')['Quantity'].sum()
      .sort_values(ascending=False).head(10))

print("\n[INFO] Trade side distribution counts:")
print(combined_df['Side'].value_counts())

# 2. Visualizations

# Distribution of trade sides
sns.countplot(data=combined_df, x='Side')
plt.title('Distribution of Trade Sides')
plt.xlabel('Trade Side')
plt.ylabel('Count')
plt.show()

# Distribution of unit prices
sns.histplot(data=combined_df, x='Unit Price', bins=50, kde=True)
plt.title('Distribution of Unit Prices')
plt.xlabel('Unit Price')
plt.ylabel('Frequency')
plt.show()

# Quantity vs Unit Price by Side
sns.scatterplot(data=combined_df, x='Quantity', y='Unit Price', hue='Side', alpha=0.5)
plt.title('Quantity vs Unit Price by Trade Side')
plt.xlabel('Quantity')
plt.ylabel('Unit Price')
plt.show()

# Daily trade volume trends
daily_volume = combined_df.groupby('Trade Date')['Quantity'].sum().reset_index()
sns.lineplot(data=daily_volume, x='Trade Date', y='Quantity')
plt.title('Daily Trade Volume Over Time')
plt.xlabel('Trade Date')
plt.ylabel('Total Quantity')
plt.show()

# Hourly trade activity trends
hourly_volume = combined_df.groupby('Trade Hour')['Quantity'].sum().reset_index()
sns.barplot(data=hourly_volume, x='Trade Hour', y='Quantity', palette='viridis')
plt.title('Trading Activity by Hour of Day')
plt.xlabel('Hour (UTC)')
plt.ylabel('Total Quantity')
plt.show()


## Step-4: OutliersDetection
- Boxplots quickly highlight extreme outliers visually.
- The IQR method identifies and removes statistically extreme points while preserving most of the valid observations.
- Comparing statistics before and after filtering quantifies the effect of outlier handling on Quantity and Unit Price.

In [ ]:
# 1. Visual examination of outliers using boxplots
print("\n[INFO] Visualizing Unit Price outliers using boxplot:")
sns.boxplot(data=combined_df, x='Unit Price')
plt.title('Unit Price Outlier Detection (Boxplot)')
plt.xlabel('Unit Price')
plt.show()

print("\n[INFO] Visualizing Quantity outliers using boxplot:")
sns.boxplot(data=combined_df, x='Quantity')
plt.title('Quantity Outlier Detection (Boxplot)')
plt.xlabel('Quantity')
plt.show()

# 2. Statistical detection using the IQR method
def detect_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    return outliers

quantity_outliers = detect_outliers_iqr(combined_df['Quantity'])
price_outliers = detect_outliers_iqr(combined_df['Unit Price'])

print(f"\n[INFO] Number of Quantity outliers detected: {len(quantity_outliers)}")
print(f"[INFO] Number of Unit Price outliers detected: {len(price_outliers)}")

# 3. Statistical comparison before and after filtering
print("\n[INFO] Summary statistics BEFORE filtering outliers:")
print(combined_df[['Quantity', 'Unit Price']].describe())

# Optionally remove outliers for clean analysis
cleaned_df = combined_df[~combined_df['Quantity'].isin(quantity_outliers)]
cleaned_df = cleaned_df[~cleaned_df['Unit Price'].isin(price_outliers)]

print("\n[INFO] Summary statistics AFTER removing outliers:")
print(cleaned_df[['Quantity', 'Unit Price']].describe())
print(f"[INFO] New dataset shape after outlier removal: {cleaned_df.shape}")


## Step-5 Venue and instrument analysis

In [ ]:
# 1. Aggregated metrics per venue
print("\n[INFO] Calculating aggregated trading metrics per venue:")
venue_summary = cleaned_df.groupby('Venue Identifier').agg({
    'Quantity': 'sum',
    'Trade Amount': 'sum',
    'Instrument Identifier': 'nunique'
}).sort_values(by='Trade Amount', ascending=False)

print("\n[INFO] Top 10 venues by total trade amount:")
print(venue_summary.head(10))

# Visualize top trading venues
sns.barplot(
    data=venue_summary.head(10).reset_index(),
    x='Venue Identifier',
    y='Trade Amount',
    palette='coolwarm'
)
plt.title('Top 10 Venues by Trade Amount')
plt.xlabel('Venue Identifier')
plt.ylabel('Total Trade Amount')
plt.xticks(rotation=45)
plt.show()

# 2. Instrument-level analysis
print("\n[INFO] Calculating key trade statistics per instrument:")
instrument_summary = cleaned_df.groupby('Instrument Identifier').agg({
    'Quantity': 'sum',
    'Trade Amount': 'sum',
    'Unit Price': 'mean',
    'Venue Identifier': 'nunique'
}).sort_values(by='Trade Amount', ascending=False)

print("\n[INFO] Top 10 instruments by total trade amount:")
print(instrument_summary.head(10))

# Visualize instrument concentration
sns.barplot(
    data=instrument_summary.head(10).reset_index(),
    x='Instrument Identifier',
    y='Trade Amount',
    palette='viridis'
)
plt.title('Top 10 Instruments by Trade Amount')
plt.xlabel('Instrument Identifier')
plt.ylabel('Total Trade Amount')
plt.xticks(rotation=45)
plt.show()

# 3. Cross-analysis between Trade Side and Venue
print("\n[INFO] Analyzing trade side distributions per venue:")
side_venue = cleaned_df.groupby(['Venue Identifier', 'Side']).size().unstack(fill_value=0)
print(side_venue.head())

side_venue.plot(kind='bar', stacked=True, figsize=(10,6), colormap='coolwarm')
plt.title('Trade Side Distribution per Venue')
plt.xlabel('Venue Identifier')
plt.ylabel('Trade Count')
plt.show()


## Step-7 Model preparation

In [ ]:
ohe = OneHotEncoder(sparse=False, drop='first')

In [ ]:
# --- Step 6: Model Preparation ---

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd

# 1. Select relevant features for modeling
print("\n[INFO] Selecting relevant modeling features:")
selected_features = ['Quantity', 'Unit Price', 'Trade Amount', 'Price Currency', 'Venue Identifier', 'Side', 'Instrument Identifier']
model_df = cleaned_df[selected_features].copy()
print(model_df.head(5))

# 2. One-hot encode categorical features
print("\n[INFO] Encoding categorical features:")
ohe = OneHotEncoder(sparse=False, drop='first')
encoded_features = pd.DataFrame(
    ohe.fit_transform(model_df[['Price Currency', 'Venue Identifier', 'Side', 'Instrument Identifier']]),
    columns=ohe.get_feature_names_out(['Price Currency', 'Venue Identifier', 'Side', 'Instrument Identifier'])
)

# Combine encoded and numeric columns
model_ready_df = pd.concat([model_df[['Quantity', 'Unit Price', 'Trade Amount']].reset_index(drop=True), encoded_features], axis=1)

print(f"[INFO] Final shape after encoding: {model_ready_df.shape}")
print("[INFO] Displaying first 5 rows of model-ready dataset:")
print(model_ready_df.head())

# 3. Split dataset for modeling
print("\n[INFO] Splitting data into training and testing sets:")
X = model_ready_df.drop('Trade Amount', axis=1)
y = model_ready_df['Trade Amount']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"[INFO] Training samples: {X_train.shape[0]}")
print(f"[INFO] Testing samples: {X_test.shape[0]}")

# 4. Scale numeric variables
print("\n[INFO] Scaling numeric features for model consistency:")
scaler = StandardScaler()
num_cols = ['Quantity', 'Unit Price']
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

print("[INFO] Model preparation completed. Data is ready for machine learning or regression analysis.")
